<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Constraining the multi-fault rupture dynamics of the Mw 6.5 Norcia earthquake, Italy</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Elisa Tinti, Emanuele Casarotti, Thomas Ulrich, Taufiqurrahman, Duo Li, Alice Agnes Gabriel</div>
        </div>
    </div>
</div>

In this notebook, we will prepare input for dynamic models using SeisSol (http://www.seissol.org) and reproduce figures shown in Tinti et. al., (2021). We will study how the rupture history on the fault planes may evolve assuming different heterogeneous distribution of dynamic parameters, and to investigate if such model can match specific features of the Norcia, 30 October 2016, Mw6.5 event.  Our main goal is to assess the mechanical viability of the complex kinematic model proposed by Scognamiglio  et  al. (2018),  S18 hereinafter, testing if there are dynamic models with the same kinematic features able to spontaneously propagate. We therefore search for ”families” of plausible dynamic models  that share similar dynamic features and are able to reproduce specific kinematic characteristics.

Let's get started!

In [ ]:
# import modules and setup 
# -------------------------------
%matplotlib inline
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from scipy import interpolate
from scipy.interpolate import interp1d
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
import glob
import pandas as pd
import json

import matplotlib as mpl
label_size = 10
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 11
mpl.rcParams['axes.titlesize'] = 13
mpl.rcParams['figure.titlesize'] = 20

import pyproj
utm33 = pyproj.Proj("+proj=utm +zone=33, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

## Fault Geometry

The fault geometry proposed in this study is derived from Scognamiglio et al. (2018) and is composed of two faults:  a main fault striking N155° trending along the Apennines and dipping 47° to  the  SW  (hereinafter  F155),  and  a  second  fault  plane  striking  N210° oblique  to the  Apennines  and  dipping  36° to  the  NW  (hereinafter  F210).   The  main  fault  is  34 km long and 16 km downdip wide, while the secondary fault is 10 km long and 14 km wide.   F155 reaches the modeled  free surface,  while  the  top  border  of  F210  is  1.8  km below  the  modeled  ground  surface.

In the local mesh reference system, fault F155 strike direction is along X-axis, Y-axis is perpendicular to X-axis and Z-axis is positive upward.

The orientation of the reference system has been defined for simplifying the orientation of the 2 faults: the x axis is parallel to the strike vector of the fault F155, the y axis is perpendicular with the positive direction in the F155 footwall space.

In [ ]:
# fault geometry F155 
# ------------------------
fault155_dip=90-47 # starting from z-
fault155_strike=0 # fault155 is oriented along x

rx1 = R.from_euler('x', -fault155_dip, degrees=True) # around X for dip 
rx2 = R.from_euler('z', fault155_strike, degrees=True) # around Z for strike
rx = rx2*rx1

n155=rx.apply([0,-1,0]) #rotation
# print('F155 normal',n155)

# fault geometry F210
# ------------------------
fault210_dip=90-36 # starting from z-
fault210_strike=-55 # fault210 strike is 155-210 where 155 is oriented along x

nx1 = R.from_euler('x', -fault210_dip, degrees=True) #around X for dip 
nx2 = R.from_euler('z', fault210_strike, degrees=True) #around Z for strike
nx = nx2*nx1

nx3 =R.from_euler('y',-5.86085404017178, degrees=True)
px=nx3*nx2*nx1
px2=nx*nx3

n210=nx.apply([0,-1,0]) #rotation 
# print('F210 normal',n210)

retta = np.cross(n210,n155)
retta = retta/np.linalg.norm(retta)
# print ('F210 dip direction',retta)

perp=np.cross(retta,n210) # find perpendicular to retta and normal to 210
perp=perp/np.linalg.norm(perp)
# print('F210 strike direction',perp)

In [ ]:
# reference coordinates
# --------------------------
origin = [348441.37745855964, 4760209.936366674, 0] # converted to UTM coordinates
hypocenter = [345545.0171306281 - origin[0],
              4744751.470962405 - origin[1],
              -9520.0]

rotz = R.from_euler('z', -90+155, degrees=True)
_hypo_meshcoord = rotz.apply(hypocenter)

epicenter = _hypo_meshcoord-[17000, 0, 0] #traslation
zhypo = -(n155[0]*epicenter[0]+n155[1]*epicenter[1])/n155[2] #depth along dip
hypo_meshcoord=np.array([epicenter[0], epicenter[1], zhypo])

firstcorner = hypo_meshcoord-500*retta
secondcorner = firstcorner+14000*retta
thirdcorner = firstcorner+10000*perp
forthcorner = secondcorner+10000*perp
hypocenter_geo = [13.11, 42.8398, -9.52*1000]

## Kinematic model

The S18 kinematic model proposed by Scognamiglio et al. (2018) has been obtained by inverting strong motion velocity waveforms  in  the  frequency  range  of  0.02  -  0.5  Hz  (Butterworth  filter with  2  passes  2  poles).

In [ ]:
# load kinematic model data
# -------------------------------
slipfile_a = 'modelloAAgeorefzeroshallowslip.out'
slipfile_b = 'modelloBBgeoref.out'

# column definition
# ---------------------
# % lon lat depth(km) x(km) y(km) dx dy slip(m) rake dip strike timerup(s) rise(s)
d_a = np.loadtxt(slipfile_a, skiprows=1, delimiter=' ',
                 usecols=[0, 1, 2, 3, 4, 7, 8])
d_b = np.loadtxt(slipfile_b, skiprows=1, delimiter=' ',
                 usecols=[0, 1, 2, 3, 4, 7, 8])

# fault geometry of F155 
# ---------------------------
dip_a = 47.
strike_a = 90.  # because the main fault F155 is along x axis

step = 200.  # in meters
nz0_a = 33
nx0_a = 69
tmp_a = np.ones([nx0_a, nz0_a])

lons_a = d_a[:, 0]
lats_a = d_a[:, 1]
depths_a = d_a[:, 2]*1000
xs_a = d_a[:, 3]
ys_a = d_a[:, 4]
slip_a = d_a[:, 5]
rake_a = d_a[:, 6]

slip_a.shape = (nx0_a, nz0_a)
lons_a.shape = (nx0_a, nz0_a)
lats_a.shape = (nx0_a, nz0_a)
xs_a.shape = (nx0_a, nz0_a)
ys_a.shape = (nx0_a, nz0_a)
depths_a.shape = (nx0_a, nz0_a)  # vertical along z coordinates
rake_a.shape = (nx0_a, nz0_a)

gsmal_a=slip_a<0.05
rake_a_mod=rake_a.copy()
rake_a_mod[gsmal_a]=-90

# fault geometry of F210 
# ---------------------------
dip_b = 36.
strike_b = 145.  # because the main fault F155 is along x axis, 210-155+90

directiondip = retta.copy()
directiondip.shape = (3, 1)
directionstrike = perp.copy()
directionstrike.shape = (3, 1)

step = 200.  # in meters
nz0_b = 29
nx0_b = 21
tmp_b = np.ones([nx0_b, nz0_b])

lons_b = d_b[:, 0]
lats_b = d_b[:, 1]
depths_b = d_b[:, 2]*1000
xs_b = d_b[:, 3]
ys_b = d_b[:, 4]
slip_b = d_b[:, 5]
rake_b = d_b[:, 6]

slip_b.shape = (nx0_b, nz0_b)
lons_b.shape = (nx0_b, nz0_b)
lats_b.shape = (nx0_b, nz0_b)
xs_b.shape = (nx0_b, nz0_b)
ys_b.shape = (nx0_b, nz0_b)
depths_b.shape = (nx0_b, nz0_b)  # vertical along z coordinates
rake_b.shape = (nx0_b, nz0_b)

gsmal_b=slip_b<0.05
rake_b_mod=rake_b.copy()
rake_b_mod[gsmal_b]=-10

# compute distance
# ---------------------
dist = np.sqrt(
    (lons_a-hypocenter_geo[0])**2+(lats_a-hypocenter_geo[1])**2+(depths_a-hypocenter_geo[2])**2)
ind_ipo = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
ind_ipo_b=(0,27)

# translate since the origin in the slip model is in the center of the subfault (2kmx2km)
# -----------------------------------------------------------------------------------------------------
xs_a = xs_a * 1000 + 1000
xs_b = xs_b * 1000 + 1000
ys_a = ys_a * 1000 - 1000
ys_b = ys_b * 1000 -1000

x_a = np.linspace(np.min(xs_a), np.max(xs_a), nx0_a)
z_a = np.linspace(np.max(ys_a), np.min(ys_a), nz0_a)
x_b = np.linspace(np.min(xs_b), np.max(xs_b), nx0_b)
z_b = np.linspace(np.max(ys_b), np.min(ys_b), nz0_b)

nx_a = len(x_a)
nz_a = len(z_a)
zv_a, xv_a = np.meshgrid(z_a, x_a)

nx_b = len(x_b)
nz_b = len(z_b)
zv_b, xv_b = np.meshgrid(z_b, x_b)

matrix_zv_b=zv_b.ravel()
matrix_zv_b.shape=(zv_b.ravel().shape[0],1)
matrix_xv_b=xv_b.ravel()
matrix_xv_b.shape=(xv_b.ravel().shape[0],1)

# z along fault ---> real depth
#  ---------------------------------
new_zv_b = secondcorner + \
            matrix_xv_b * directionstrike.T + \
            matrix_zv_b*directiondip.T

depth_z_b = new_zv_b[:, 2]
depth_z_b.shape = zv_b.shape

depth_z_a = zv_a*np.sin(np.radians(dip_a))

# conversion to UTM 33 coordinates
# ----------------------------------------
utmx_a,utmy_a=utm33(lons_a,lats_a)
#print(utmx_a[0][0],utmy_a[0][0])
#print(utmx_a[ind_ipo[0]][ind_ipo[1]],utmy_a[ind_ipo[0]][ind_ipo[1]],depths_a[ind_ipo[0]][ind_ipo[1]])
utmx_b,utmy_b=utm33(lons_b,lats_b)
#print(utmx_b[0][0],utmy_b[0][0])

labellat=np.arange(42.7,43,0.1)
labely=[utm33(13.019618,l)[1] for l in labellat]

labellon=np.arange(13.0,13.4,0.1)
labelx=[utm33(l,42.6607246)[0] for l in labellon]

# fault polygon for figures
# -----------------------------
a1=utmx_a==utmx_a.min()
a2=utmy_a==utmy_a.min()
a4=utmy_a==utmy_a.max()
a3=utmx_a==utmx_a.max()

b1=utmx_b==utmx_b.min()
b2=utmy_b==utmy_b.min()
b4=utmy_b==utmy_b.max()
b3=utmx_b==utmx_b.max()

poly_b=np.array([utmx_b[b1],utmy_b[b1],
                 utmx_b[b2],utmy_b[b2],
                 utmx_b[b3],utmy_b[b3],
                 utmx_b[b4],utmy_b[b4]] )
poly_b.shape=(4,2)

poly_a=np.array([utmx_a[a1],utmy_a[a1],
                 utmx_a[a2],utmy_a[a2],
                 utmx_a[a3],utmy_a[a3],
                 utmx_a[a4],utmy_a[a4]] )
poly_a.shape=(4,2)

poly_c=np.array([utmx_b[b1]-10000,utmy_b[b1]-7000,
                 utmx_b[b2]-10000,utmy_b[b2]-7000,
                 utmx_b[b3]-10000,utmy_b[b3]-7000,
                 utmx_b[b4]-10000,utmy_b[b4]-7000] )
poly_c.shape=(4,2)

## Families of Initial Dynamic Parameters

Dynamic models can be built assuming homogeneous or heterogeneous spatial distri-bution of one or more dynamic parameters on the fault plane (Savran and Olsen 2020). In the framework of a linear slip-weakening law (LSW), a spontaneous dynamic modelis fully defined by the spatial distributions of initial on-fault shear traction (magnitude and orientation), normal traction, static and dynamic friction, and Dc parameter.

To limit the number of degree-of-freedom and therefore the complexity of the parameterization, it is common to find in the literature dynamic models that attribute all the heterogeneities only to the initial stress distribution assuming constant μs and μd or to the yield strength ( in terms of μs) while considering the other dynamic parameter homogeneous.

## FAMILY H

Homogeneous family is based on homogeneous stress conditions, may be useful to understand which conditions may favor or not a spontaneous multi-fault rupture. We assume homogeneous static and dynamic friction coefficients equalto 0.6 and 0.2, respectively.  These are standard values frequently adopted in literatureand consistent with laboratory experiments for most of the lithological structures. 

Furthermore, we set the initial stress τo as 65% of the yield stress τy, which practically allows the rupture to spontaneously propagate at sub-shear velocity.  Among various as-sumptions  required  to  fully  define  a  dynamic  model,  the  choice  of  the  direction  (rake) of the initial stress strongly influences the evolution of the rupture history.  In general, initial stress is assumed to be oriented in the direction of final slip to keep shear tractioncollinear with inferred slip velocity (Tinti et al.  2005a), which is a condition commonly used in dynamic modelling (Cocco et al 2016).  In these homogeneous models we fix the rake constant on the fault planes assuming the average rake orientation of the faults in the original S18 model:  pure normal faulting for the F155 (−90) and almost pure left-lateral strike faulting (−10) for the F210.

Therefore, normal and shear traction on these simple models vary on the two faultplanes only as a function of depth because normal stress is depth-dependent. We also shows the variation along a cross section of τy, τo and τd.  The change of the slope in the depth range of 5-10 km is the result of the layered density profile.

In [ ]:
# family H parameters
# -------------------------
NOME_SIMULAZIONE='H_family'

# dynamic friction (𝜇𝑑), static friction (𝜇𝑠)
# -------------------------------------------------
MU_D = 0.2  # dynamic friction
MU_S = 0.6  # static friction
MU_TR = 10.  # maximum static friction

# Dc, slip weaking critical distance
# ----------------------------------------
D_CR_b=0.6*tmp_b # Dc is a constant % of total slip
D_CR_a=1.2*tmp_a # Dc is a constant % of total slip

# density
# --------
DENSITY_METHOD = 'vertical_depth_dependent'

gravity = 9.8  # [m/s^2]
rho_val = 2700  # [kg/m^3]

# density profile of the 1D nnCIA profile Herrmann et al. (2011)
z_nncia = [-1500., -4500, -7500, -14500, 
           -29500, -35500, -43500, -100000]  # [meters]
rho_nncia = [2275.3, 2485.8, 2705.8, 2609.3,
             2677., 3000.2, 3012., 3206.]  # [kg/m^3]

rho_b = rho_val*tmp_b
rho_a = rho_val*tmp_a

for zrho, rho_val in zip(z_nncia[::-1], rho_nncia[::-1]):
    # print(zrho,rho_val,zrho/np.sin(dip*np.pi/180))
    criterium = depth_z_a >= zrho
    rho_a[criterium] = rho_val
    criterium = depth_z_b >= zrho
    rho_b[criterium] = rho_val


rho_a = rho_a-1000  # adding the presence of fluid
rho_b = rho_b-1000  # adding the presence of fluid

# normal stress (𝜎𝑛)
# ---------------------
NORMAL_STRESS_METHOD='vertical_depth_dependent'

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_a = -rho_a*gravity*depth_z_a/1e6 # [MPa]
NORMAL_STRESS_uni_a = -rho_val*gravity*depth_z_a/1e6 # [MPa]

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_b = -rho_b*gravity*depth_z_b/1e6 # [MPa]
NORMAL_STRESS_uni_b = -rho_val*gravity*depth_z_b/1e6 # [MPa]

sn_thre = 1  # [MPa]
NORMAL_STRESS_a[NORMAL_STRESS_a < sn_thre] = sn_thre
NORMAL_STRESS_b[NORMAL_STRESS_b < sn_thre] = sn_thre

# initial stress (𝜏𝑜) , dynamic frictional stress (𝜏𝑓=𝜇𝑑𝜎𝑛), yield stress (𝜏𝑦)
# --------------------------------------------------------------------------------------
TAU_Y_a = NORMAL_STRESS_a*MU_S
TAU_Y_b = NORMAL_STRESS_b*MU_S

TAU_F_a = NORMAL_STRESS_a*MU_D
TAU_F_b = NORMAL_STRESS_b*MU_D

# 𝜏𝑜 is set 63% of 𝜏𝑦=𝜇𝑠𝜎𝑛 before increasing 𝜏𝑦 close to the surface
# -------------------------------------------------------------------------------
TAU_0_a = TAU_Y_a*0.63
TAU_0_b = TAU_Y_b*0.63

mu_s_a = TAU_Y_a/NORMAL_STRESS_a
mu_s_a = np.minimum(mu_s_a, MU_TR)
TAU_Y_a = mu_s_a*NORMAL_STRESS_a

mu_s_b = TAU_Y_b/NORMAL_STRESS_b
mu_s_b = np.minimum(mu_s_b, MU_TR)
TAU_Y_b = mu_s_b*NORMAL_STRESS_b

mu_d_a = TAU_F_a/NORMAL_STRESS_a
mu_d_b = TAU_F_b/NORMAL_STRESS_b

stress_a=TAU_0_a
stress_b=TAU_0_b

R_a=np.round((stress_a-TAU_F_a)/(TAU_Y_a-TAU_F_a),1)
R_b=np.round((stress_b-TAU_F_b)/(TAU_Y_b-TAU_F_b),1)

# rotate stress tensor
# ----------------------
def rotate(sxx, syy, sxy, sxz, syz, szz=0, thetax=0, thetay=0, thetaz=0):
    rad_thetax = np.radians(thetax)
    rad_thetay = np.radians(thetay)
    rad_thetaz = np.radians(thetaz)
    cx = np.cos(rad_thetax)
    cy = np.cos(rad_thetay)
    cz = np.cos(rad_thetaz)
    sx = np.sin(rad_thetax)
    sy = np.sin(rad_thetay)
    sz = np.sin(rad_thetaz)

    Tx = np.array([[1, 0, 0],
                      [0, cx, -sx],
                      [0, sx,  cx]])
    Ty = np.array([[cy, 0, -sy],
                      [0, 1, 0],
                      [sy, 0, cy]])
    Tz = np.array([[cz, -sz, 0],
                      [sz, cz, 0],
                      [0, 0, 1]])

    A = np.array([[sxx, sxy, sxz],
                     [sxy, syy, syz],
                     [sxz, syz,  szz]])

    A_tempx = np.matmul(np.matmul(Tx, A), np.transpose(Tx))
    A_tempy = np.matmul(np.matmul(Ty, A_tempx), np.transpose(Ty))
    Stress_in_global_xyz = np.matmul(
        np.matmul(Tz, A_tempy), np.transpose(Tz))

    return A, Stress_in_global_xyz

def rotating_stress_tensor(stress_a, stress_b,rake_a=None,rake_b=None):
    s_yy_a = 0.*tmp_a.ravel()
    s_zz_a = 0.*tmp_a.ravel()
    s_yz_a = 0.*tmp_a.ravel()
    s_xx_a = 0.*tmp_a.ravel()
    s_xz_a = 0.*tmp_a.ravel()
    s_xy_a = 0.*tmp_a.ravel()
    
    if rake_a is not None:
        stress_a_0=stress_a*np.cos(np.radians(rake_a))
        stress_a_90=-stress_a*np.sin(np.radians(rake_a))
    else:
        stress_a_0=stress_a
        stress_a_90=stress_a
    
    for i,(s0,s90,sn) in enumerate(zip(stress_a_0.ravel(),stress_a_90.ravel(),NORMAL_STRESS_a.ravel())):
        _,snew=rotate(sxx=0,syy=-sn*1e6,sxy=-s0*1e6,sxz=0,syz=s90*1e6,szz=0,thetax=dip_a-90,thetay=0,thetaz=90-strike_a)

        s_yy_a[i] = snew[1, 1]
        s_zz_a[i] = snew[2, 2]
        s_yz_a[i] = snew[2, 1]
        s_xx_a[i] = snew[0, 0]
        s_xz_a[i] = snew[2, 0]
        s_xy_a[i] = snew[1, 0]

    s_yy_a.shape = stress_a.shape
    s_zz_a.shape = stress_a.shape
    s_yz_a.shape = stress_a.shape
    s_xx_a.shape = stress_a.shape
    s_xz_a.shape = stress_a.shape
    s_xy_a.shape = stress_a.shape

    s_yy_b = 0.*tmp_b.ravel()
    s_zz_b = 0.*tmp_b.ravel()
    s_yz_b = 0.*tmp_b.ravel()
    s_xx_b = 0*tmp_b.ravel()
    s_xz_b = 0*tmp_b.ravel()
    s_xy_b = 0*tmp_b.ravel()
    
    if rake_b is not None:
        stress_b_0=stress_b*np.cos(np.radians(rake_b))
        stress_b_90=-stress_b*np.sin(np.radians(rake_b))
    else:
        stress_b_0=stress_b
        stress_b_90=stress_b
        
    for i,(s0b,s90b,sn) in enumerate(zip(stress_b_0.ravel(),stress_b_90.ravel(),NORMAL_STRESS_b.ravel())):
        _,snew=rotate(sxx=0,syy=-sn*1e6,sxy=-s0b*1e6,sxz=0,syz=s90b*1e6,szz=0,thetax=dip_b-90,thetay=0,thetaz=90-strike_b) # beware of the negative sign of sn
        
    # beware of the negative sign of sn
        s_yy_b[i] = snew[1, 1]
        s_zz_b[i] = snew[2, 2]
        s_yz_b[i] = snew[2, 1]
        s_xx_b[i] = snew[0, 0]
        s_xz_b[i] = snew[2, 0]
        s_xy_b[i] = snew[1, 0]

    s_yy_b.shape = stress_b.shape
    s_zz_b.shape = stress_b.shape
    s_yz_b.shape = stress_b.shape
    s_xx_b.shape = stress_b.shape
    s_xz_b.shape = stress_b.shape
    s_xy_b.shape = stress_b.shape
    
    return s_yy_a, s_zz_a, s_yz_a, s_xx_a, s_xz_a, s_xy_a, s_yy_b, s_zz_b, s_yz_b, s_xx_b, s_xz_b, s_xy_b

s_yy_a, s_zz_a, s_yz_a, s_xx_a, s_xz_a, s_xy_a, s_yy_b, s_zz_b, s_yz_b, s_xx_b, s_xz_b, s_xy_b = rotating_stress_tensor(stress_a, stress_b)

In [ ]:
# plot FAMILY H 
# ------------------
def plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='magma_r',
                         fmat="%.1f", shiftx=10000, shifty=7000, colors=None,
                         quiver=False,profile=False):
    
    title=txt.split('[')[0]
    try:
        umis='['+txt.split('[')[1]
    except:
        umis=''
    if v_a is not None and quiver is False:
        cont = ax.contourf(utmx_a, utmy_a, v_a, levels=50,
                           vmin=vmin, vmax=vmax, cmap=cmap)
        cont2 = ax.contourf(utmx_b-shiftx, utmy_b-shifty, v_b, levels=50,
                            alpha=1, vmin=vmin, vmax=vmax, cmap=cmap)
    elif quiver is False:
        cont = ax.contourf(utmx_a, utmy_a, utmx_a*0, levels=50,
                           vmin=vmin, vmax=vmax, cmap=cmap)
        cont2 = ax.contourf(utmx_b-shiftx, utmy_b-shifty, utmx_b*0, levels=50,
                            alpha=1, vmin=vmin, vmax=vmax, cmap=cmap)

    patches = []
    polygon = Polygon(poly_a, True)
    patches.append(polygon)
    polygon = Polygon(poly_c, True)
    patches.append(polygon)
    p = PatchCollection(patches, facecolor='none',
                        edgecolor='k', linewidth=2, alpha=1)
    ax.add_collection(p)

    patches = []
    polygon = Polygon(poly_b, True)
    patches.append(polygon)
    px = [utmx_b[b2]-shiftx, utmx_b[b2]]
    py = [utmy_b[b2]-shifty, utmy_b[b2]]
    line = mlines.Line2D(px, py)
    patches.append(line)
    px = [utmx_b[b3]-shiftx, utmx_b[b3]]
    py = [utmy_b[b3]-shifty, utmy_b[b3]]
    line = mlines.Line2D(px, py)
    patches.append(line)
    px = [utmx_b[b4]-shiftx, utmx_b[b4]]
    py = [utmy_b[b4]-shifty, utmy_b[b4]]
    line = mlines.Line2D(px, py)
    patches.append(line)
    p = PatchCollection(patches, facecolor='none',
                        edgecolor='k', linewidth=1, alpha=0.6)
    ax.add_collection(p)

    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    if v_a is not None:
        if type(cmap) == matplotlib.colors.ListedColormap:
            if len(colors) == 2:
                length = (vmax-vmin)/2.
                clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ticks=[
                    vmin+length/2., vmax-length/2.], spacing='proportional', format=fmat, orientation='horizontal')
                clb.ax.set_xticklabels([vmin, vmax])
            elif len(colors) == 1:
                clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ticks=[
                    vmin], spacing='proportional', format=fmat, orientation='horizontal')
                clb.ax.set_xticklabels([round(vmin,1)])
    
        else:
            clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ticks=np.linspace(
                vmin, vmax, 3), spacing='proportional', format=fmat, orientation='horizontal')
        clb.set_label(umis)
    else:
        clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ticks=[],
                           spacing='proportional', format=fmat, orientation='horizontal')
        clb.outline.set_visible(False)
        
    if quiver:
        decimation=5
        U = np.cos(np.radians(-65+rake_a))
        V = np.sin(np.radians(-65+rake_a))
        ax.quiver(utmx_a[::decimation, ::decimation], 
                  utmy_a[::decimation, ::decimation], 
                  U[::decimation, ::decimation], V[::decimation, ::decimation],
                  color='k',
                  width=.026,alpha=1,cmap=cmap,norm=norm)
        ax.quiver(utmx_a[::decimation, ::decimation], 
                  utmy_a[::decimation, ::decimation], 
                  U[::decimation, ::decimation], V[::decimation, ::decimation],
                  v_a[::decimation, ::decimation],
                  width=.025,alpha=1,cmap=cmap,norm=norm)
        U = np.cos(np.radians(-120+rake_b))
        V = np.sin(np.radians(-120+rake_b))
        ax.quiver(utmx_b[::decimation, ::decimation]-shiftx, 
                  utmy_b[::decimation, ::decimation]-shifty, 
                  U[::decimation, ::decimation], V[::decimation, ::decimation],
                  color='k',
                  width=.026,alpha=1,cmap=cmap,norm=norm)
        ax.quiver(utmx_b[::decimation, ::decimation]-shiftx, 
                  utmy_b[::decimation, ::decimation]-shifty, 
                  U[::decimation, ::decimation], V[::decimation, ::decimation],
                  v_b[::decimation, ::decimation],
                  width=.025,alpha=1,cmap=cmap,norm=norm)
    
    if profile:
        ax.plot(utmx_a[l,:],utmy_a[l,:],'r--',lw=2)
    
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.set_xticks(labelx)
    ax.set_xticklabels([round(xl, 2) for xl in labellon])
    ax.set_xlim(utmx_a.min()-shiftx, utmx_a.max())
    ax.set_yticks(labely)
    ax.set_yticklabels([round(xl, 2) for xl in labellat])
    ax.set_axis_off()
    
# plot parameters
# -------------------
fig = plt.figure(figsize=(20, 15))
plt.tight_layout()

icol = 6
irow = 3
l=ind_ipo[0]

i = 1
ax = fig.add_subplot(irow, icol, i)
v_a = None  # slip_a
v_b = None  # slip_b
vmin = 0  # min(v_a.min(),v_b.min())
vmax = 0  # max(v_a.max(),v_b.max())
txt = 'slip [m]'
colors = ['white']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 2
ax = fig.add_subplot(irow, icol, i)
v_a = rho_a
v_b = rho_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = 'density [kg/$m^3$] '+DENSITY_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax,
                         txt, cmap='plasma', fmat="%.0f")

i = 3
ax = fig.add_subplot(irow, icol, i)
v_a = D_CR_a
v_b = D_CR_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$D_c$ [m]'
colors = ['red', 'green']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 4
ax = fig.add_subplot(irow, icol, i)
v_a = NORMAL_STRESS_a
v_b = NORMAL_STRESS_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\sigma_n$ [MPa] '+NORMAL_STRESS_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis')


i = 5
ax = fig.add_subplot(irow, icol, i)
v_a = mu_s_a
v_b = mu_s_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\mu_s$'
colors = ['blue']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 6
ax = fig.add_subplot(irow, icol, i)
v_a = mu_d_a
v_b = mu_d_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\mu_d$'
colors = ['green']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 7
ax = fig.add_subplot(irow, icol, i)
v_a = None
v_b = None
vmin = 0# min(v_a.min(), v_b.min())
vmax = 0# max(v_a.max(), v_b.max())
txt = 'slip -> rake [m]'
colors = ['white']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 8
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_0_a
v_b = TAU_0_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_{o} = \\tau_{f} + c_{slip \\rightarrow stress}*slip$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 9
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_F_a
v_b = TAU_F_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_f = \\mu_{d} \\sigma_{n}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 12
ax = fig.add_subplot(irow, icol, i)
v_a = R_a
v_b = R_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
# print(vmin, vmax)
txt = '$R = \\frac{\\tau_o - \\tau_f }{\\tau_y - \\tau_f}$'
colors = ['red']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)


v=np.concatenate([s_yy_a.ravel(), s_zz_a.ravel(), s_yz_a.ravel(), s_xx_a.ravel(), 
                s_xz_a.ravel(), s_xy_a.ravel(), s_yy_b.ravel(), s_zz_b.ravel(),
                s_yz_b.ravel(), s_xx_b.ravel(), s_xz_b.ravel(), s_xy_b.ravel()])

vmin=-max(abs(v.min()/1e6),abs(v.max()/1e6))
vmax=max(abs(v.min()/1e6),abs(v.max()/1e6))
# print(vmin,vmax)
cmap='seismic'

i = 13
ax = fig.add_subplot(irow, icol, i)
v_a = s_xx_a/1e6
v_b = s_xx_b/1e6
txt = '$\sigma_{xx}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 14
ax = fig.add_subplot(irow, icol, i)
v_a = s_yy_a/1e6
v_b = s_yy_b/1e6
txt = '$\sigma_{yy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 15
ax = fig.add_subplot(irow, icol, i)
v_a = s_zz_a/1e6
v_b = s_zz_b/1e6
txt = '$\sigma_{zz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 16
ax = fig.add_subplot(irow, icol, i)
v_a = s_xy_a/1e6
v_b = s_xy_b/1e6
txt = '$\sigma_{xy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 17
ax = fig.add_subplot(irow, icol, i)
v_a = s_xz_a/1e6
v_b = s_xz_b/1e6
txt = '$\sigma_{xz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 18
ax = fig.add_subplot(irow, icol, i)
v_a = s_yz_a/1e6
v_b = s_yz_b/1e6
txt = '$\sigma_{yz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 10
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_Y_a
v_b = TAU_Y_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = ' $\\tau_y = \\tau_o + \\Delta S$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 11
ax1 = fig.add_subplot(irow, icol, i)
ax1.plot(TAU_0_a[l,:],zv_a[l,:],label='tau_o',color='k')
ax1.plot(TAU_Y_a[l,:],zv_a[l,:],label='tau_y',color='r')
ax1.plot(TAU_F_a[l,:],zv_a[l,:],label='tau_f',color='b')
ax1.legend(frameon=False)
ax1.yaxis.tick_right()
ax1.set_yticks([0,-5000,-10000,-15000])
ax1.set_yticklabels(str(_) for _ in [0,-5,-10,-15])
ax1.set_ylabel('along dip [km]')
ax1.set_xlabel('[MPa]]')
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=plt.cm.colors.ListedColormap(['white'])), ticks=[],
                           spacing='proportional', format='%.1f', orientation='horizontal')
clb.outline.set_visible(False)
from matplotlib.patches import ConnectionPatch
xy=(utmx_a[l,0],utmy_a[l,0])
xy1=(0,1)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)
xy=(utmx_a[l,-1],utmy_a[l,-1])
xy1=(0,0)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)

plt.suptitle('H FAMILY')
plt.show()

## FAMILY B

Family  B  represents  the  group  of  heterogeneous  models  frequently  used by dynamic modeler. The stress drop is set heterogeneous, correlated to the S18 inferred  slip  distribution.   The  yield  strength  is  assumed  heterogeneous  to allow a variable as well as small strength excess all over the fault plane. This  assumption appears reasonable when compared with the stress directly computed from the slipdistribution using a static mechanical procedure The yield strength is assumed heterogeneous to allow a variableas well as small strength excess all over the fault plane.  In these models, spontaneous rupture propagation is favored everywhere on the fault planes and leads to a heterogeneous slip distribution.

In [ ]:
# family B parameters
# -------------------------
NOME_SIMULAZIONE='B_family'

# dynamic friction (𝜇𝑑), static friction (𝜇𝑠)
# -------------------------------------------------
MU_D = 0.2  # dynamic friction
MU_S = 0.5  # static friction
MU_TR = 10.  # maximum static friction

# Dc, slip weaking critical distance
# ----------------------------------------
# 𝐷𝑐  is a percentage of the total slip, the percentage depends on the depth
z_nncia=[-1500.,-4500,-7500,-14500,-29500,-35500,-43500,-100000]
dc_nncia=[0.3,0.3,0.1,0.1] 
D_CR_tres=0.02
D_CR_b=0.6*tmp_b 
D_CR_a=0.6*tmp_a 

for zdc,dc_val in zip(z_nncia[len(dc_nncia)-1::-1],dc_nncia[::-1]):
    #print(zdc,dc_val)
    cdc = depth_z_a >= zdc
    D_CR_a[cdc] = dc_val
    cdc = depth_z_b >= zdc
    D_CR_b[cdc] = dc_val
    
D_CR_a=D_CR_a*slip_a
cdc = D_CR_a <= D_CR_tres
D_CR_a[cdc]=D_CR_tres 

D_CR_b=D_CR_b*slip_b
cdc = D_CR_b <= D_CR_tres
D_CR_b[cdc]=D_CR_tres

# density
# --------
DENSITY_METHOD = 'vertical_depth_dependent'
rho_b = rho_val*tmp_b
rho_a = rho_val*tmp_a

for zrho, rho_val in zip(z_nncia[::-1], rho_nncia[::-1]):
    # print(zrho,rho_val,zrho/np.sin(dip*np.pi/180))
    criterium = depth_z_a >= zrho
    rho_a[criterium] = rho_val
    criterium = depth_z_b >= zrho
    rho_b[criterium] = rho_val


rho_a = rho_a-1000  # adding the presence of fluid
rho_b = rho_b-1000  # adding the presence of fluid

# normal stress (𝜎𝑛)
# ---------------------
NORMAL_STRESS_METHOD='vertical_depth_dependent'

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_a = -rho_a*gravity*depth_z_a/1e6 # [MPa]
NORMAL_STRESS_uni_a = -rho_val*gravity*depth_z_a/1e6 # [MPa]

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_b = -rho_b*gravity*depth_z_b/1e6 # [MPa]
NORMAL_STRESS_uni_b = -rho_val*gravity*depth_z_b/1e6 # [MPa]

sn_thre = 1  # [MPa]
NORMAL_STRESS_a[NORMAL_STRESS_a < sn_thre] = sn_thre
NORMAL_STRESS_b[NORMAL_STRESS_b < sn_thre] = sn_thre

# initial stress (𝜏𝑜) , dynamic frictional stress (𝜏𝑓=𝜇𝑑𝜎𝑛), yield stress (𝜏𝑦)
# --------------------------------------------------------------------------------------
SLIP_2_STRESS_FACTOR_a=4
SLIP_2_STRESS_FACTOR_b=5
TAU_F_a = NORMAL_STRESS_a*MU_D
TAU_F_b = NORMAL_STRESS_b*MU_D

# 𝜏𝑜  is set 𝜏𝑜=𝜏𝑓+𝑐𝑠𝑙𝑖𝑝 → 𝑠𝑡𝑟𝑒𝑠𝑠∗𝑠𝑙𝑖𝑝
# ---------------------------------------------
TAU_0_a=TAU_F_a+slip_a*SLIP_2_STRESS_FACTOR_a
TAU_0_b=TAU_F_b+slip_b*SLIP_2_STRESS_FACTOR_b

# Strengh excess  Δ𝑆  is defined as a function of the distance from the hypocenter  
# -----------------------------------------------------------------------------------------------
# Δ𝑆 = ((𝜏𝑚𝑎𝑥𝑦−𝜏𝑚𝑖𝑛𝑦)/𝑟𝑐) ∗ 𝑟 + 𝜏𝑚𝑖𝑛𝑦  with  𝜏𝑚𝑖𝑛𝑦 = 0.1  and 𝜏𝑚𝑎𝑥𝑦 = 0.6
def get_tauy(xx, yy, xxc, yyc, rr_crit=80000, tauy_min=0.1, tauy_max=6):
    ffy=np.ones(xx.shape)*tauy_max
    r=np.sqrt((xx-xxc)**2+(yy-yyc)**2)
    c=r <= rr_crit
    ffy[c]=(tauy_max-tauy_min)/rr_crit*r[c]+tauy_min
    return ffy

STREXCE_a=get_tauy(xv_a,zv_a,xv_a[ind_ipo],zv_a[ind_ipo])
STREXCE_b=get_tauy(xv_b,zv_b,xv_b[ind_ipo_b],zv_b[ind_ipo_b])

# 𝜏𝑦  is set 𝜏𝑦=𝜏𝑜+Δ𝑆
# -------------------------
TAU_Y_a=TAU_0_a+STREXCE_a
TAU_Y_b=TAU_0_b+STREXCE_b

mu_s_a = TAU_Y_a/NORMAL_STRESS_a
mu_s_a = np.minimum(mu_s_a, MU_TR)
TAU_Y_a = mu_s_a*NORMAL_STRESS_a

mu_s_b = TAU_Y_b/NORMAL_STRESS_b
mu_s_b = np.minimum(mu_s_b, MU_TR)
TAU_Y_b = mu_s_b*NORMAL_STRESS_b

mu_d_a = TAU_F_a/NORMAL_STRESS_a
mu_d_b = TAU_F_b/NORMAL_STRESS_b

stress_a=TAU_0_a
stress_b=TAU_0_b

R_a=np.round((stress_a-TAU_F_a)/(TAU_Y_a-TAU_F_a),1)
R_b=np.round((stress_b-TAU_F_b)/(TAU_Y_b-TAU_F_b),1)

# rotate stress tensor
# ----------------------
s_yy_a, s_zz_a, s_yz_a, s_xx_a, s_xz_a, s_xy_a, s_yy_b, s_zz_b, s_yz_b, s_xx_b, s_xz_b, s_xy_b = rotating_stress_tensor(stress_a, stress_b,rake_a_mod,rake_b_mod)

In [ ]:
# plot FAMILY B 
# ------------------
fig = plt.figure(figsize=(20, 15))
plt.tight_layout()

icol = 6
irow = 3
l=ind_ipo[0]

i = 1
ax = fig.add_subplot(irow, icol, i)
v_a = slip_a
v_b = slip_b
vmin = min(v_a.min(),v_b.min())
vmax = max(v_a.max(),v_b.max())
txt = 'slip [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='hot_r')

i = 2
ax = fig.add_subplot(irow, icol, i)
v_a = rho_a
v_b = rho_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = 'density [kg/$m^3$] '+DENSITY_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax,
                         txt, cmap='plasma', fmat="%.0f")

i = 3
ax = fig.add_subplot(irow, icol, i)
v_a = D_CR_a
v_b = D_CR_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$D_c$ [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='RdYlGn')

i = 4
ax = fig.add_subplot(irow, icol, i)
v_a = NORMAL_STRESS_a
v_b = NORMAL_STRESS_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\sigma_n$ [MPa] '+NORMAL_STRESS_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis')

i = 5
ax = fig.add_subplot(irow, icol, i)
v_a = mu_s_a
v_b = mu_s_b
vmin = min(v_a.min(), v_b.min())
vmax = 1# max(v_a.max(), v_b.max())
txt = '$\mu_s$'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                        cmap='Blues')

i = 6
ax = fig.add_subplot(irow, icol, i)
v_a = mu_d_a
v_b = mu_d_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\mu_d$'
colors = ['green']
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap=plt.cm.colors.ListedColormap(colors), colors=colors)

i = 7
ax = fig.add_subplot(irow, icol, i)
v_a = slip_a
v_b = slip_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = 'slip -> rake [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='hot_r',quiver=True)

i = 8
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_0_a
v_b = TAU_0_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_{o} = \\tau_{f} + c_{slip \\rightarrow stress}*slip$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 9
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_F_a
v_b = TAU_F_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_f = \\mu_{d} \\sigma_{n}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 12
ax = fig.add_subplot(irow, icol, i)
v_a = .5*D_CR_a*(TAU_0_a-TAU_F_a)
v_b = .5*D_CR_b*(TAU_0_b-TAU_F_b)
vmin = min(v_a.min(), v_b.min())
vmax = 10 #max(v_a.max(), v_b.max())

txt = '$Fracture Energy = \\frac{\\tau_{o} - \\tau_{f} }{2}D_{c}$'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='Reds')

v=np.concatenate([s_yy_a.ravel(), s_zz_a.ravel(), s_yz_a.ravel(), s_xx_a.ravel(), 
                s_xz_a.ravel(), s_xy_a.ravel(), s_yy_b.ravel(), s_zz_b.ravel(),
                s_yz_b.ravel(), s_xx_b.ravel(), s_xz_b.ravel(), s_xy_b.ravel()])

vmin=-max(abs(v.min()/1e6),abs(v.max()/1e6))
vmax=max(abs(v.min()/1e6),abs(v.max()/1e6))
cmap='seismic'

i = 13
ax = fig.add_subplot(irow, icol, i)
v_a = s_xx_a/1e6
v_b = s_xx_b/1e6
txt = '$\sigma_{xx}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 14
ax = fig.add_subplot(irow, icol, i)
v_a = s_yy_a/1e6
v_b = s_yy_b/1e6
txt = '$\sigma_{yy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 15
ax = fig.add_subplot(irow, icol, i)
v_a = s_zz_a/1e6
v_b = s_zz_b/1e6
txt = '$\sigma_{zz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 16
ax = fig.add_subplot(irow, icol, i)
v_a = s_xy_a/1e6
v_b = s_xy_b/1e6
txt = '$\sigma_{xy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 17
ax = fig.add_subplot(irow, icol, i)
v_a = s_xz_a/1e6
v_b = s_xz_b/1e6
txt = '$\sigma_{xz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 18
ax = fig.add_subplot(irow, icol, i)
v_a = s_yz_a/1e6
v_b = s_yz_b/1e6
txt = '$\sigma_{yz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 10
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_Y_a
v_b = TAU_Y_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = ' $\\tau_y = \\tau_o + \\Delta S$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 11
ax1 = fig.add_subplot(irow, icol, i)
ax1.plot(TAU_0_a[l,:],zv_a[l,:],label='tau_o',color='k')
ax1.plot(TAU_Y_a[l,:],zv_a[l,:],label='tau_y',color='r')
ax1.plot(TAU_F_a[l,:],zv_a[l,:],label='tau_f',color='b')
ax1.legend(frameon=False)
ax1.yaxis.tick_right()
ax1.set_yticks([0,-5000,-10000,-15000])
ax1.set_yticklabels(str(_) for _ in [0,-5,-10,-15])
ax1.set_ylabel('along dip [km]')
ax1.set_xlabel('[MPa]]')
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=plt.cm.colors.ListedColormap(['white'])), ticks=[],
                           spacing='proportional', format='%.1f', orientation='horizontal')
clb.outline.set_visible(False)
from matplotlib.patches import ConnectionPatch
xy=(utmx_a[l,0],utmy_a[l,0])
xy1=(0,1)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)
xy=(utmx_a[l,-1],utmy_a[l,-1])
xy1=(0,0)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)

plt.suptitle('B FAMILY')
plt.show()

## FAMILY C

Family  C  includes  models  with  homogeneous  static  friction  and  initial  stress,  and heterogeneous dynamic friction, with values compatible with the expected rocks in this region and at these depths.  This family of models is motivated by the fact that mostrocks share similar $\mu_s$.  Heterogeneities in the dynamic friction coefficient allow constraining the target stress drop distribution, which is again assumed correlated to the inferred slip distribution.

In [ ]:
# family C parameters
# -------------------------
NOME_SIMULAZIONE='C_family'

# dynamic friction (𝜇𝑑), static friction (𝜇𝑠)
# -------------------------------------------------
MU_D = 0.1  # dynamic friction
MU_S = 0.6  # static friction
MU_TR = 10.  # maximum static friction

# Dc, slip weaking critical distance
# ----------------------------------------
# 𝐷𝑐  is a percentage of the total slip, the percentage depends on the depth
z_nncia=[-1500.,-4500,-7500,-14500,-29500,-35500,-43500,-100000]
dc_nncia=[0.3,0.3,0.1,0.1]
dc_nncia_add=[1.,0.,0.,0.]
D_CR_tres=0.02
D_CR_b=0.6*tmp_b 
D_CR_a=0.6*tmp_a 
D_CR_b2=0.*tmp_b 
D_CR_a2=0.*tmp_a 

for zdc,dc_val,dc_add in zip(z_nncia[len(dc_nncia)-1::-1],dc_nncia[::-1],dc_nncia_add[::-1]):
    #print(zdc,dc_val)
    cdc = depth_z_a >= zdc
    D_CR_a[cdc] = dc_val
    D_CR_a2[cdc] = dc_add
    cdc = depth_z_b >= zdc
    D_CR_b[cdc] = dc_val
    D_CR_b2[cdc] = dc_add
    
D_CR_a=D_CR_a*slip_a+D_CR_a2
cdc = D_CR_a <= D_CR_tres
D_CR_a[cdc]=D_CR_tres 

D_CR_b=D_CR_b*slip_b+D_CR_b2
cdc = D_CR_b <= D_CR_tres
D_CR_b[cdc]=D_CR_tres

# density
# --------
DENSITY_METHOD = 'vertical_depth_dependent'
rho_b = rho_val*tmp_b
rho_a = rho_val*tmp_a

for zrho, rho_val in zip(z_nncia[::-1], rho_nncia[::-1]):
    criterium = depth_z_a >= zrho
    rho_a[criterium] = rho_val
    criterium = depth_z_b >= zrho
    rho_b[criterium] = rho_val

rho_a = rho_a-1000  # adding the presence of fluid
rho_b = rho_b-1000  # adding the presence of fluid

# normal stress (𝜎𝑛)
# ---------------------
NORMAL_STRESS_METHOD='vertical_depth_dependent'

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_a = -rho_a*gravity*depth_z_a/1e6 # [MPa]
NORMAL_STRESS_uni_a = -rho_val*gravity*depth_z_a/1e6 # [MPa]

# zv is negative but NORMAL_STRESS is positive in compression
NORMAL_STRESS_b = -rho_b*gravity*depth_z_b/1e6 # [MPa]
NORMAL_STRESS_uni_b = -rho_val*gravity*depth_z_b/1e6 # [MPa]

sn_thre = 1  # [MPa]
NORMAL_STRESS_a[NORMAL_STRESS_a < sn_thre] = sn_thre
NORMAL_STRESS_b[NORMAL_STRESS_b < sn_thre] = sn_thre

# initial stress (𝜏𝑜) , dynamic frictional stress (𝜏𝑓=𝜇𝑑𝜎𝑛), yield stress (𝜏𝑦)
# --------------------------------------------------------------------------------------
slip_a_thres=slip_a.copy()
slip_b_thres=slip_b.copy()

Slip_thres=0.5
tsli = slip_a_thres <= Slip_thres
slip_a_thres[tsli]=Slip_thres
tsli = slip_b_thres <= Slip_thres
slip_b_thres[tsli]=Slip_thres

# 𝜏𝑦  is set 𝜏𝑦=𝜇𝑠𝜎𝑛 then the condition 𝜏𝑦 >= 5 is applied
# --------------------------------------------------------------------
TAU_Y_a=NORMAL_STRESS_a*MU_S
TAU_Y_b=NORMAL_STRESS_b*MU_S

TauY_tres=5
ttauY = TAU_Y_a <= TauY_tres
TAU_Y_a[ttauY]=TauY_tres
ttauY = TAU_Y_b <= TauY_tres
TAU_Y_b[ttauY]=TauY_tres

mu_s_a = TAU_Y_a/NORMAL_STRESS_a
mu_s_b = TAU_Y_b/NORMAL_STRESS_b

mu_s_a=np.minimum(mu_s_a,MU_TR)
TAU_Y_a=mu_s_a*NORMAL_STRESS_a
mu_s_b=np.minimum(mu_s_b,MU_TR)
TAU_Y_b=mu_s_b*NORMAL_STRESS_b

# 𝜏𝑜  is set 𝜏𝑜=𝑐𝜇𝑠𝜎𝑛 where 𝑐=0.80 for F155 and 𝑐=0.85 for F210
# -----------------------------------------------------------------------------
TAU_0_a=NORMAL_STRESS_a*MU_S*0.80
TAU_0_b=NORMAL_STRESS_b*MU_S*0.85

# 𝜏𝑓  is set 𝜏𝑓=𝜏𝑜−𝑐𝑠𝑙𝑖𝑝 → 𝑠𝑡𝑟𝑒𝑠𝑠∗𝑠𝑙𝑖𝑝
# ---------------------------------------------
SLIP_2_STRESS_FACTOR_a=5
SLIP_2_STRESS_FACTOR_b=8

TAU_F_a=TAU_0_a-slip_a_thres*SLIP_2_STRESS_FACTOR_a
TAU_F_b=TAU_0_b-slip_b_thres*SLIP_2_STRESS_FACTOR_b

mu_d_a=TAU_F_a/NORMAL_STRESS_a
mu_d_b=TAU_F_b/NORMAL_STRESS_b

tmu = mu_d_a <= MU_D
mu_d_a[tmu]= MU_D

tmu = mu_d_b <= MU_D
mu_d_b[tmu]= MU_D

TAU_F_a=NORMAL_STRESS_a*mu_d_a
TAU_F_b=NORMAL_STRESS_b*mu_d_b

mu_d_a = TAU_F_a/NORMAL_STRESS_a
mu_d_b = TAU_F_b/NORMAL_STRESS_b

stress_a=TAU_0_a
stress_b=TAU_0_b

R_a=np.round((stress_a-TAU_F_a)/(TAU_Y_a-TAU_F_a),1)
R_b=np.round((stress_b-TAU_F_b)/(TAU_Y_b-TAU_F_b),1)

# rotate stress tensor
# ----------------------
s_yy_a, s_zz_a, s_yz_a, s_xx_a, s_xz_a, s_xy_a, s_yy_b, s_zz_b, s_yz_b, s_xx_b, s_xz_b, s_xy_b = rotating_stress_tensor(stress_a, stress_b, rake_a_mod, rake_b_mod)

In [ ]:
# plot FAMILY C 
# ------------------
fig = plt.figure(figsize=(20, 15))
plt.tight_layout()

icol = 6
irow = 3
l=ind_ipo[0]

i = 1
ax = fig.add_subplot(irow, icol, i)
v_a = slip_a
v_b = slip_b
vmin = min(v_a.min(),v_b.min())
vmax = max(v_a.max(),v_b.max())
txt = 'slip [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='hot_r')

i = 2
ax = fig.add_subplot(irow, icol, i)
v_a = rho_a
v_b = rho_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = 'density [kg/$m^3$] '+DENSITY_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax,
                         txt, cmap='plasma', fmat="%.0f")

i = 3
ax = fig.add_subplot(irow, icol, i)
v_a = D_CR_a
v_b = D_CR_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$D_c$ [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='RdYlGn')

i = 4
ax = fig.add_subplot(irow, icol, i)
v_a = NORMAL_STRESS_a
v_b = NORMAL_STRESS_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\sigma_n$ [MPa] '+NORMAL_STRESS_METHOD
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis')


i = 5
ax = fig.add_subplot(irow, icol, i)
v_a = mu_s_a
v_b = mu_s_b
vmin = min(v_a.min(), v_b.min())
vmax = 1# max(v_a.max(), v_b.max())
txt = '$\mu_s$'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                        cmap='Blues')

i = 6
ax = fig.add_subplot(irow, icol, i)
v_a = mu_d_a
v_b = mu_d_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\mu_d$'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='Greens')

i = 7
ax = fig.add_subplot(irow, icol, i)
v_a = slip_a
v_b = slip_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = 'slip -> rake [m]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='hot_r',quiver=True)


i = 8
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_0_a
v_b = TAU_0_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_{o} = c \\mu_s \\sigma_n$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)


i = 9
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_F_a
v_b = TAU_F_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = '$\\tau_{f} = \\tau_{o} - c_{slip \\rightarrow stress}*slip$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 12
ax = fig.add_subplot(irow, icol, i)
v_a = .5*D_CR_a*(TAU_0_a-TAU_F_a)
v_b = .5*D_CR_b*(TAU_0_b-TAU_F_b)
vmin = min(v_a.min(), v_b.min())
vmax = 10 #max(v_a.max(), v_b.max())
txt = '$Fracture Energy = \\frac{\\tau_{o} - \\tau_{f} }{2}D_{c}$'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt,
                         cmap='Reds')

v=np.concatenate([s_yy_a.ravel(), s_zz_a.ravel(), s_yz_a.ravel(), s_xx_a.ravel(), 
                s_xz_a.ravel(), s_xy_a.ravel(), s_yy_b.ravel(), s_zz_b.ravel(),
                s_yz_b.ravel(), s_xx_b.ravel(), s_xz_b.ravel(), s_xy_b.ravel()])

vmin=-max(abs(v.min()/1e6),abs(v.max()/1e6))
vmax=max(abs(v.min()/1e6),abs(v.max()/1e6))
cmap='seismic'

i = 13
ax = fig.add_subplot(irow, icol, i)
v_a = s_xx_a/1e6
v_b = s_xx_b/1e6
txt = '$\sigma_{xx}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 14
ax = fig.add_subplot(irow, icol, i)
v_a = s_yy_a/1e6
v_b = s_yy_b/1e6
txt = '$\sigma_{yy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 15
ax = fig.add_subplot(irow, icol, i)
v_a = s_zz_a/1e6
v_b = s_zz_b/1e6
txt = '$\sigma_{zz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 16
ax = fig.add_subplot(irow, icol, i)
v_a = s_xy_a/1e6
v_b = s_xy_b/1e6
txt = '$\sigma_{xy}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 17
ax = fig.add_subplot(irow, icol, i)
v_a = s_xz_a/1e6
v_b = s_xz_b/1e6
txt = '$\sigma_{xz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 18
ax = fig.add_subplot(irow, icol, i)
v_a = s_yz_a/1e6
v_b = s_yz_b/1e6
txt = '$\sigma_{yz}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap=cmap)

i = 10
ax = fig.add_subplot(irow, icol, i)
v_a = TAU_Y_a
v_b = TAU_Y_b
vmin = min(v_a.min(), v_b.min())
vmax = max(v_a.max(), v_b.max())
txt = ' $\\tau_y = \\mu_{s} \\sigma_{n}$ [MPa]'
d = plot_topview_2faults(ax, v_a, v_b, vmin, vmax, txt, cmap='viridis',profile=True)

i = 11
ax1 = fig.add_subplot(irow, icol, i)
ax1.plot(TAU_0_a[l,:],zv_a[l,:],label='tau_o',color='k')
ax1.plot(TAU_Y_a[l,:],zv_a[l,:],label='tau_y',color='r')
ax1.plot(TAU_F_a[l,:],zv_a[l,:],label='tau_f',color='b')
ax1.legend(frameon=False)
ax1.yaxis.tick_right()
ax1.set_yticks([0,-5000,-10000,-15000])
ax1.set_yticklabels(str(_) for _ in [0,-5,-10,-15])
ax1.set_ylabel('along dip [km]')
ax1.set_xlabel('[MPa]]')
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
clb = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=plt.cm.colors.ListedColormap(['white'])), ticks=[],
                           spacing='proportional', format='%.1f', orientation='horizontal')
clb.outline.set_visible(False)

from matplotlib.patches import ConnectionPatch
xy=(utmx_a[l,0],utmy_a[l,0])
xy1=(0,1)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)
xy=(utmx_a[l,-1],utmy_a[l,-1])
xy1=(0,0)
con = ConnectionPatch(
    xyA=xy, coordsA=ax.transData,
    xyB=xy1, coordsB=ax1.transAxes,
    arrowstyle="-", shrinkB=5,color='r')
ax1.add_artist(con)

plt.suptitle('C FAMILY')
plt.show()

In [ ]:
# create 3D NetCDF input files for F155
# ----------------------------------------------
fname155='norciax_155fault_nncia.nc'
fname155_pw='paraview_'+fname155

def extrapolate_nans(x, y, v):
    if np.ma.is_masked(v):
        nans = v.mask
    else:
        nans = np.isnan(v)
    notnans = np.logical_not(nans)
    v[nans] = interpolate.griddata((x[notnans], y[notnans]), v[notnans], (x[nans], y[nans]), method='nearest').ravel()
    return v

xcornerA = np.asarray([-17000,-17000,17000,17000])
ycornerA = np.asarray([-10911.973761,0,0,-10911.973761])

stepx = 100
stepy = 50
stepz=2
x_155 = np.linspace(xcornerA.min(),xcornerA.max(),stepx)
y_155 = np.linspace(ycornerA.min(),ycornerA.max(),stepy)
xi_155,yi_155 = np.meshgrid(x_155,y_155)

xp155=[]
yp155=[]
zp155=[]
for _x,_z,_s in zip(xv_a.ravel(),zv_a.ravel(),slip_a.ravel()):
    vector=rx.apply([_x,0,_z])-[17000,0,0] #by elisa 2 tentativo
    xp155.append(vector[0])
    yp155.append(vector[1])
    zp155.append(vector[2])

xp155=np.array(xp155)
yp155=np.array(yp155)
zp155=np.array(zp155)

xp155.shape=slip_a.shape
yp155.shape=slip_a.shape

x_155 = np.linspace(xp155.min(),xp155.max(),stepx)
y_155 = np.linspace(yp155.min(),yp155.max(),stepy)
z_155 = np.linspace(zp155.min(),zp155.max(),stepz)

xi_155,yi_155 = np.meshgrid(x_155,y_155)

def create_projection(v,stepx=100,stepy=50,stepz=2):
    v = extrapolate_nans(
        xi_155,yi_155,
        interpolate.griddata((xp155.ravel(),yp155.ravel()), v.ravel(),(xi_155,yi_155),method='linear'))
    v2 = np.tile((v.ravel()),stepz)
    v2.shape=(stepz,stepy,stepx)
    return v,v2

slip_p_155,slip_p_155_0=create_projection(slip_a)

zi_155=slip_p_155_0[0,:,:]*0+1
zi_155_1=zi_155*-11701.65923
zi_155_2=zi_155*0

# plot projection 
# -----------------
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(xi_155, yi_155, zi_155_1 , c=slip_p_155.ravel(), marker='.',alpha=0.1)
# ax.scatter(xi_155, yi_155, zi_155_2 , c=slip_p_155.ravel(), marker='.',alpha=0.1)

# write NetCDF file for ASAGI
# -----------------------------------
fout = Dataset('ex_'+fname155,'w',format='NETCDF4')
fout.createDimension('x',stepx)
fout.createDimension('y',stepy)
fout.createDimension('z',stepz)
fout.createVariable('x',np.float32,('x',))
fout.createVariable('y',np.float32,('y',))
fout.createVariable('z',np.float32,('z',))
fout.variables['x'][:] = x_155;
fout.variables['y'][:] = y_155;
fout.variables['z'][:] = z_155;

initial_condition = np.dtype([('s_zz',np.dtype('f4')),
                              ('s_yy',np.dtype('f4')),
                              ('s_yz',np.dtype('f4')),
                              ('s_xx',np.dtype('f4')),
                              ('s_xz',np.dtype('f4')),
                              ('s_xy',np.dtype('f4')),
                              ('d_c',np.dtype('f4')),
                              ('mu_s',np.dtype('f4'))])

s_zz_p,s_zz_p_0=create_projection(s_zz_a)
s_yy_p,s_yy_p_0=create_projection(s_yy_a)
s_yz_p,s_yz_p_0=create_projection(s_yz_a)
s_xx_p,s_xx_p_0=create_projection(s_xx_a)
s_xz_p,s_xz_p_0=create_projection(s_xz_a)
s_xy_p,s_xy_p_0=create_projection(s_xy_a)
d_c_p,d_c_p_0=create_projection(D_CR_a)
mu_s_p,mu_s_p_0=create_projection(mu_s_a)

data = fout.createCompoundType(initial_condition,'initial_condition')
vv = fout.createVariable('data',data,dimensions=('z', 'y', 'x'))

for k in range(0,stepz):
    for jj in range (stepy):
        for i in range(0,stepx):
            vv[k,jj,i] = (s_zz_p_0[k,jj,i],s_yy_p_0[k,jj,i],s_yz_p_0[k,jj,i],s_xx_p_0[k,jj,i],
                          s_xz_p_0[k,jj,i],s_xy_p_0[k,jj,i],d_c_p_0[k,jj,i],mu_s_p_0[k,jj,i])
            
fout.close()

# write NetCDF file for Paraview
# --------------------------------------
fnetcdf = Dataset('ex_'+fname155_pw, 'w', format='NETCDF3_64BIT')

fnetcdf.createDimension('x', stepx)
fnetcdf.createDimension('y', stepy)
fnetcdf.createDimension('z', stepz)

desc = 'test'
fnetcdf.description = desc

x_nc = fnetcdf.createVariable('x', 'f4', ('x'))
y_nc = fnetcdf.createVariable('y', 'f4', ('y'))
z_nc = fnetcdf.createVariable('z', 'f4', ('z'))

s_yz_nc = fnetcdf.createVariable('s_yz', 'f4', ('z', 'y','x'))
s_yy_nc = fnetcdf.createVariable('s_yy', 'f4', ('z', 'y','x'))
s_zz_nc = fnetcdf.createVariable('s_zz', 'f4', ('z', 'y','x'))
s_xx_nc = fnetcdf.createVariable('s_xx', 'f4', ('z', 'y','x'))
s_xz_nc = fnetcdf.createVariable('s_xz', 'f4', ('z', 'y','x'))
s_xy_nc = fnetcdf.createVariable('s_xy', 'f4', ('z', 'y','x'))
dc_nc = fnetcdf.createVariable('d_c', 'f4', ('z', 'y','x'))
mu_s_nc = fnetcdf.createVariable('mu_s', 'f4', ('z', 'y','x'))

s_zz_nc[:,:,:] = s_zz_p_0
s_yy_nc[:,:,:] = s_yy_p_0
s_yz_nc[:,:,:] = s_yz_p_0
s_xx_nc[:,:,:] = s_xx_p_0
s_xz_nc[:,:,:] = s_xz_p_0
s_xy_nc[:,:,:] = s_xy_p_0
dc_nc[:,:,:] = d_c_p_0
mu_s_nc[:,:,:] = mu_s_p_0

x_nc[:] = x_155
y_nc[:] = y_155
z_nc[:] = z_155
fnetcdf.close()

In [ ]:
# create 3D NetCDF input files for F210 
# ----------------------------------------------
fname210='norciax_210fault_nncia.nc'
fname210_pw='paraview_'+fname210

xcorner = np.asarray([-4514.264329707738, 
                3895.1754241867684, 
                10277.668240782505, 
                1868.2284868879979])
ycorner = np.asarray([-9430.650446469763, 
                -1797.1014780883834, 
                -9471.965636164652, 
                -17105.51460454603])

stepx = 80
stepy = 80
x_210 = np.linspace(xcorner.min(),xcorner.max(),stepx)
y_210 = np.linspace(ycorner.min(),ycorner.max(),stepy)
xi_210,yi_210 = np.meshgrid(x_210,y_210)

xp210=[]
yp210=[]
zp210=[]
for _x,_z,_s in zip(xv_b.ravel(),zv_b.ravel(),slip_b.ravel()):
    vector=px2.apply([_x,0,_z])+secondcorner
    xp210.append(vector[0])
    yp210.append(vector[1])
    zp210.append(vector[2])

xp210=np.array(xp210)
yp210=np.array(yp210)
zp210=np.array(zp210)

xp210.shape=slip_b.shape
yp210.shape=slip_b.shape

x_210 = np.linspace(xp210.min(),xp210.max(),stepx)
y_210 = np.linspace(yp210.min(),yp210.max(),stepy)
z_210 = np.linspace(zp210.min(),zp210.max(),stepz)
xi_210,yi_210 = np.meshgrid(x_210,y_210)

def create_projection(v,stepx=80,stepy=80,stepz=2):
    v = extrapolate_nans(
        xi_210,yi_210,
        interpolate.griddata((xp210.ravel(),yp210.ravel()), v.ravel(),(xi_210,yi_210),method='linear'))
    v2 = np.tile((v.ravel()),stepz)
    v2.shape=(stepz,stepy,stepx) #questo ordine controlatto da casarotti (anche se y e x hanno stessa dimensione va prima la y)
    return v,v2

slip_p_210,slip_p_210_0=create_projection(slip_b)
stepz = 2 # only top and bottom

zi_210=slip_p_210_0[0,:,:]*0+1
zi_210_1=zi_210*-10113.134453974475
zi_210_2=zi_210*-1326.9507610865114

# plot projection 
# -----------------
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(xi_210, yi_210, zi_210_1 , c=slip_p_210.ravel(), marker='.',alpha=0.1)
# ax.scatter(xi_210, yi_210, zi_210_2 , c=slip_p_210.ravel(), marker='.',alpha=0.1)
# plt.show()

# write NetCDF file for ASAGI
# -----------------------------------
fout = Dataset('ex_'+fname210,'w',format='NETCDF4')
fout.createDimension('x',stepx)
fout.createDimension('y',stepy)
fout.createDimension('z',stepz)
fout.createVariable('x',np.float32,('x',))
fout.createVariable('y',np.float32,('y',))
fout.createVariable('z',np.float32,('z',))
fout.variables['x'][:] = x_210;
fout.variables['y'][:] = y_210;
fout.variables['z'][:] = z_210;

initial_condition = np.dtype([('s_zz',np.dtype('f4')),
                              ('s_yy',np.dtype('f4')),
                              ('s_yz',np.dtype('f4')),
                              ('s_xx',np.dtype('f4')),
                              ('s_xz',np.dtype('f4')),
                              ('s_xy',np.dtype('f4')),
                              ('d_c',np.dtype('f4')),
                              ('mu_s',np.dtype('f4'))])

s_zz_p,s_zz_p_0=create_projection(s_zz_b)
s_yy_p,s_yy_p_0=create_projection(s_yy_b)
s_yz_p,s_yz_p_0=create_projection(s_yz_b)
s_xx_p,s_xx_p_0=create_projection(s_xx_b)
s_xz_p,s_xz_p_0=create_projection(s_xz_b)
s_xy_p,s_xy_p_0=create_projection(s_xy_b)
d_c_p,d_c_p_0=create_projection(D_CR_b)
mu_s_p,mu_s_p_0=create_projection(mu_s_b)

data = fout.createCompoundType(initial_condition,'initial_condition')
vv = fout.createVariable('data',data,dimensions=('z', 'y', 'x'))


for k in range(0,stepz):
    for jj in range (stepy):
        for i in range(0,stepx):
            vv[k,jj,i] = (s_zz_p_0[k,jj,i],s_yy_p_0[k,jj,i],s_yz_p_0[k,jj,i],s_xx_p_0[k,jj,i],
                          s_xz_p_0[k,jj,i],s_xy_p_0[k,jj,i],d_c_p_0[k,jj,i],mu_s_p_0[k,jj,i])
fout.close()

# write NetCDF file for Paraview
# --------------------------------------
fnetcdf = Dataset('ex_'+fname210_pw, 'w', format='NETCDF3_64BIT')

fnetcdf.createDimension('x', stepx)
fnetcdf.createDimension('y', stepy)
fnetcdf.createDimension('z', stepz)

desc = 'test'
fnetcdf.description = desc

x_nc = fnetcdf.createVariable('x', 'f4', ('x'))
y_nc = fnetcdf.createVariable('y', 'f4', ('y'))
z_nc = fnetcdf.createVariable('z', 'f4', ('z'))

s_yz_nc = fnetcdf.createVariable('s_yz', 'f4', ('z', 'y','x'))
s_yy_nc = fnetcdf.createVariable('s_yy', 'f4', ('z', 'y','x'))
s_zz_nc = fnetcdf.createVariable('s_zz', 'f4', ('z', 'y','x'))
s_xx_nc = fnetcdf.createVariable('s_xx', 'f4', ('z', 'y','x'))
s_xz_nc = fnetcdf.createVariable('s_xz', 'f4', ('z', 'y','x'))
s_xy_nc = fnetcdf.createVariable('s_xy', 'f4', ('z', 'y','x'))
dc_nc = fnetcdf.createVariable('d_c', 'f4', ('z', 'y','x'))
mu_s_nc = fnetcdf.createVariable('mu_s', 'f4', ('z', 'y','x'))

s_zz_nc[:,:,:] = s_zz_p_0
s_yy_nc[:,:,:] = s_yy_p_0
s_yz_nc[:,:,:] = s_yz_p_0
s_xx_nc[:,:,:] = s_xx_p_0
s_xz_nc[:,:,:] = s_xz_p_0
s_xy_nc[:,:,:] = s_xy_p_0
dc_nc[:,:,:] = d_c_p_0
mu_s_nc[:,:,:] = mu_s_p_0

x_nc[:] = x_210
y_nc[:] = y_210
z_nc[:] = z_210
fnetcdf.close()